In [ ]:
import pandas as pd
import numpy as np
import locale
import os

# **1. Ursprüngliche Daten laden**

# Check if files exist
required_files = ["umsatzdaten_gekuerzt.csv", "kiwo.csv", "wetter.csv", "Data Import and Preparation/feiertage.csv"]
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"File '{file}' not found.")

# Load data
umsatzdaten = pd.read_csv("umsatzdaten_gekuerzt.csv")
kiwo_data = pd.read_csv("kiwo.csv")
wetter_data = pd.read_csv("wetter.csv")
feiertage = pd.read_csv("Data Import and Preparation/feiertage.csv")  # Contains columns 'Datum' and 'Feiertag'

# **2. Daten vorverarbeiten**

# Convert 'Datum' to datetime for all datasets
umsatzdaten['Datum'] = pd.to_datetime(umsatzdaten['Datum'])
kiwo_data['Datum'] = pd.to_datetime(kiwo_data['Datum'])
wetter_data['Datum'] = pd.to_datetime(wetter_data['Datum'])

# Set locale to German for date parsing
try:
    locale.setlocale(locale.LC_TIME, 'de_DE.UTF-8')
except locale.Error:
    print("Locale setting to 'de_DE.UTF-8' failed. Proceeding without it.")

# Parse 'Datum' in feiertage dataset
try:
    feiertage['Datum'] = pd.to_datetime(
        feiertage['Datum'] + " 2025",  # Append year
        format='%d. %b %Y',  # Format: '1. Jan 2025'
        errors='coerce'  # Handle errors gracefully
    )
except Exception as e:
    print(f"Error parsing 'Datum' in feiertage: {e}")
    # Use fallback with manual mapping
    month_mapping = {
        "Jan": "01", "Feb": "02", "Mär": "03", "Apr": "04",
        "Mai": "05", "Jun": "06", "Jul": "07", "Aug": "08",
        "Sep": "09", "Okt": "10", "Nov": "11", "Dez": "12"
    }
    feiertage['Datum'] = feiertage['Datum'].str.replace('.', '').replace(month_mapping, regex=True)
    feiertage['Datum'] = pd.to_datetime(feiertage['Datum'] + " 2025", format='%d %m %Y')

# Drop rows with unparseable dates
feiertage = feiertage.dropna(subset=['Datum'])

# **3. Daten zusammenführen**

# Merge datasets on 'Datum'
merged_data = umsatzdaten.merge(kiwo_data, on='Datum', how='left')
merged_data = merged_data.merge(wetter_data, on='Datum', how='left')
merged_data = merged_data.merge(feiertage, on='Datum', how='left')

# **4. Zusätzliche Variablen erstellen**

# Add weekday name
merged_data['Wochentag'] = merged_data['Datum'].dt.day_name()

# Add numeric month
merged_data['Monat'] = merged_data['Datum'].dt.month


# Add binary holiday flag
merged_data['Ist_Feiertag'] = merged_data['Name'].notnull().astype(int)

# Add quarter
merged_data['Quartal'] = merged_data['Datum'].dt.quarter

# Add seasonal variable
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Frühling"
    elif month in [6, 7, 8]:
        return "Sommer"
    else:
        return "Herbst"

merged_data['Saison'] = merged_data['Monat'].apply(get_season)

# Add weekend flag
merged_data['Ist_Wochenende'] = merged_data['Wochentag'].isin(['Saturday', 'Sunday']).astype(int)

# **5. Aufbereitung der Variablen**

# Handle missing values
merged_data.fillna(0, inplace=True)

# **6. Exportieren des finalen Datensatzes**

# Create output directory if it doesn't exist
output_dir = "data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Export merged data
merged_data.to_csv(f"{output_dir}/zusammengefuehrt.csv", index=False)

# **7. Vorschau auf den finalen Datensatz**
print(merged_data.head())


       Datum  Warengruppe      Umsatz  KielerWoche  Bewoelkung  Temperatur  \
0 2013-07-01            1  148.828353          0.0         6.0     17.8375   
1 2013-07-02            1  159.793757          0.0         3.0     17.3125   
2 2013-07-03            1  111.885594          0.0         7.0     21.0750   
3 2013-07-04            1  168.864941          0.0         7.0     18.8500   
4 2013-07-05            1  171.280754          0.0         5.0     19.9750   

   Windgeschwindigkeit  Wettercode  Feiertag  Name  Art  Details  Wochentag  \
0                 15.0        20.0         0     0    0        0     Monday   
1                 10.0         0.0         0     0    0        0    Tuesday   
2                  6.0        61.0         0     0    0        0  Wednesday   
3                  7.0        20.0         0     0    0        0   Thursday   
4                 12.0         0.0         0     0    0        0     Friday   

   Monat  Ist_Feiertag  Quartal  Saison  Ist_Wochenende 

C:\Users\ayham\AppData\Local\Temp\ipykernel_22596\2161205918.py:95: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_data.fillna(0, inplace=True)
